In [85]:
import numpy as np
import scipy as sp
import sympy as smp

In [86]:
#Messwerte:
#Massen[kg]:
m_Scheibe=1085.8e-3
m_Rad=1736.3e-3
error_m=0.05e-3

#Längen[m]:
d_Scheibe=17.45e-2
a_Rad=[9.97e-2, 10.115e-2]


error_d_a=0.05e-3

#Zeiten:

#Scheibe:
t_Scheibe_tors=[28.42, 28.26, 28.30, 28.17, 28.44, 28.09, 28.15, 28.28, 28.36, 28.29]

t_Rad_tors=[30.15, 30.48, 30.21, 30.04, 30.08, 30.17, 30.13, 30.13, 30.09, 30.16, 30.15]

t_Rad_ph=[19.76, 20.36, 19.44, 19.11, 19.52, 19.83, 19.56, 19.64, 19.56, 19.49]


g=8.8070





In [87]:
#Berechnung D*:

#Trägheitsmoment Scheibe
J_Scheibe=0.5 * m_Scheibe * (d_Scheibe/2)**2
print(J_Scheibe)

#Die Dauer einer Schwingung:
T_Scheibe=np.mean(t_Scheibe_tors)/30
print(T_Scheibe)
#D*

D= (J_Scheibe * (2*np.pi)**2)/T_Scheibe**2
print(D)





0.00413286018125
0.9425333333333334
0.18366102894929276


In [88]:
#Berechnung J des Rads aus Torsions

#Dauer einer Schwingung:
T_Rad=np.mean(t_Rad_tors)/30
print(T_Rad)


J_Rad=(T_Rad/(2*np.pi))**2 * D

print(J_Rad)


1.0054242424242423
0.00470279428241196


In [89]:
#Berechnung J des Rads aus pysischen Pendel

#Abstand Achse Sp

s=np.mean(a_Rad)/2
print(s)

#Dauer der Schwingung:

T_ph=np.mean(t_Rad_ph)/30
print(T_ph)
#Trägheitsmoment
J_Rad_ph=(T_ph/(2*np.pi))**2 * m_Rad* g * s - m_Rad*s**2
print(J_Rad_ph)

0.0502125
0.6542333333333334
0.003947006147856382


In [90]:
#Fehlerrechnung 
mean_t_Scheibe=np.mean(t_Scheibe_tors)
error_t_Scheibe=np.std(t_Scheibe_tors, ddof=1)/(len(t_Scheibe_tors)*30)

print(mean_t_Scheibe)
print(error_t_Scheibe)


mean_t_Rad=np.mean(t_Rad_tors)
error_t_Rad=np.std(t_Rad_tors, ddof=1)/(len(t_Rad_tors)*30)

print(mean_t_Rad)
print(error_t_Rad)


mean_t_ph=np.mean(t_Rad_ph)
error_t_ph=np.std(t_Rad_ph, ddof=1)/(len(t_Rad_ph)*30)

print(mean_t_ph)
print(error_t_ph)





28.276000000000003
0.0003807724442774039
30.16272727272727
0.00034896959936922084
19.627000000000002
0.0010760065629245902


In [91]:
#Fehler Scheibe

def error_J_S(xm_s, em_s, xd, xed):
    m, em, d, ed=smp.symbols('m em d ed' )
    J=0.5 * m* (d/2)**2
    dJ_dm = smp.diff(J, m)  # Partielle Ableitung von J nach m
    dJ_dd = smp.diff(J, d)  # Partielle Ableitung von J nach d
    
    # Fehlerfortpflanzung
    error_J = smp.sqrt((dJ_dm * em)**2 + (dJ_dd * ed)**2)
    
    error_J_value = error_J.evalf(subs={m: xm_s, em: em_s, d: xd, ed: xed})
    
    return error_J_value*10

# Berechnung durchführen
error_J_Scheibe = error_J_S(m_Scheibe, error_m, d_Scheibe, error_d_a)


print(f"J: ± {error_J_Scheibe:.4f}")


J: ± 0.0000


In [92]:
#Fehler D

def error_D(J_Scheibe, eJ_Scheibe, T_Scheibe, eT_Scheibe):
    # Definieren der Symbole
    J, eJ, T, eT = smp.symbols('J eJ T eT')
    
    # Definiere die Formel für D
    D = J * (2 * smp.pi)**2 / T**2
    
    # Berechnung der partiellen Ableitungen
    dD_dJ = smp.diff(D, J)  # Partielle Ableitung von D nach J
    dD_dT = smp.diff(D, T)  # Partielle Ableitung von D nach T
    
    # Fehlerfortpflanzung
    error_D = smp.sqrt((dD_dJ * eJ)**2 + (dD_dT * eT)**2)
    
    # Evaluieren der Fehlerfortpflanzung
    error_D_value = error_D.evalf(subs={J: J_Scheibe, eJ: eJ_Scheibe, T: T_Scheibe, eT: eT_Scheibe})
    
    return error_D_value

D_error=error_D(J_Scheibe, error_J_Scheibe, T_Scheibe, error_t_Scheibe)
print(D_error)

0.00106626782735946


In [93]:
def error_JRad(T_Rad, eT_Rad, xD, xeD):
    # Definieren der Symbole
    T, eT, D, eD = smp.symbols('T eT D eD')
    
    # Definiere die Formel für J_Rad
    J_Rad = (T / (2 * smp.pi))**2 * D
    
    # Berechnung der partiellen Ableitungen
    dJ_dT = smp.diff(J_Rad, T)  # Partielle Ableitung von J_Rad nach T
    dJ_dD = smp.diff(J_Rad, D)  # Partielle Ableitung von J_Rad nach D
    
    # Fehlerfortpflanzung
    error_JRad = smp.sqrt((dJ_dT * eT)**2 + (dJ_dD * eD)**2)
    
    # Evaluieren der Fehlerfortpflanzung
    error_JRad_value = error_JRad.evalf(subs={T: T_Rad, eT: eT_Rad, D: xD, eD: xeD})
    
    return error_JRad_value

error_J_rad=error_JRad(T_Rad, error_t_Rad, D, D_error) 
print(error_J_rad)

2.74971570895510e-5
